In [ ]:
import h5py
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import requests
from PIL import Image
import time
import torch
import h5py
import glob
import cv2
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision import transforms as T
from torch.optim.lr_scheduler import ExponentialLR
from tqdm import tqdm
! pip install dalle-pytorch
!pip install pytorch-lightning==1.8.4
from dalle_pytorch import __version__
from torch.optim import Adam
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid, save_image
from pathlib import Path
from dalle_pytorch import OpenAIDiscreteVAE, DiscreteVAE, DALLE, VQGanVAE
ImageFile.LOAD_TRUNCATED_IMAGES = True
from tqdm.notebook import tqdm

In [ ]:
train_dataset = h5py.File('/kaggle/input/443-datasets/eee443_project_dataset_train.h5','r')

train_cap = train_dataset["train_cap"][()]
train_imid = train_dataset["train_imid"][()]
train_url = train_dataset["train_url"][()]
word_code = train_dataset["word_code"][()]


test_dataset = h5py.File('/kaggle/input/443-datasets/eee443_project_dataset_test.h5','r')
test_cap = test_dataset["test_caps"][()]
test_imid = test_dataset["test_imid"][()]
test_url=test_dataset["test_url"][()]

In [ ]:
train_img_dir="/kaggle/input/443-train-images/443images"
test_img_dir="/kaggle/input/443-test-images/443test_images"

In [1]:
2e-4

0.0002

In [ ]:
train_imid2=np.load("/kaggle/input/cleaned-data/trainimid.npy") 
train_cap2=np.load("/kaggle/input/cleaned-data/traicap.npy")
test_imid2=np.load("/kaggle/input/cleaned-data/testimid.npy")
test_cap2=np.load("/kaggle/input/cleaned-data/testcap.npy")

In [ ]:

class Create_Dataset(Dataset):

  def __init__(self, imid, cap, dir_path, **kwargs):

   self.dir_path = dir_path
   self.sample_n = len(imid)

   self.imid = imid
   self.cap = cap

  def __len__(self):

    return self.sample_n

  def __getitem__(self, ind):

     imgpath = self.dir_path + "/" + str(self.imid[ind]) + ".png"
     self.image = Image.open(imgpath).convert("RGB")

     data_tensor = self.ImageToTensor(self.image)

     return (data_tensor, self.cap[ind])

  def ImageToTensor(self, img):

    transform_with_resize = transforms.Compose([
         transforms.Resize((256, 256)),
         transforms.ToTensor(),
         transforms.Normalize((0.485,0.456,0.406), (0.229,0.224,0.225))
    ])
    get_image = transforms.Compose([transforms.ToPILImage()])
    img_rsz = transform_with_resize(img)
    tns_to_img = get_image(img_rsz)

    if img_rsz.shape[0]>3:
     img_rsz = img_rsz[:3]
    return img_rsz

  def get_splits(self):

   n_tra = int(0.85 * self.sample_n)
   n_val = self.sample_n-n_tra
    
   self.get_train, self.get_val = torch.utils.data.random_split(self, [n_tra, n_val])


In [ ]:
train_set = Create_Dataset(train_imid2, train_cap2, train_img_dir)
test_set = Create_Dataset(test_imid2, test_cap2, test_img_dir)
torch.cuda.empty_cache()

In [ ]:
train_set.get_splits()
validation_data = train_set.get_val
train_data = train_set.get_train

In [ ]:
print(len(train_data))
print(len(validation_data))

In [ ]:
parameters=dict(
    dim = 256,  
    num_text_tokens = 1004, 
    text_seq_len = 17,     
    depth = 2,    
    heads = 8,     
    dim_head = 64,    
    attn_dropout = 0.05,  
    ff_dropout = 0.05,
    reversible= False,
    stable= False,
    shift_tokens= False,
    rotary_emb= False,
    share_input_output_emb= False)       

In [ ]:

def model_save(path, opt,hparams, epoch=0):
 
    save_obj = {

              'hparams': hparams,

              'epoch': epoch,

              'version': __version__,

              'vae_class_name': "VQGanVAE",

              'weights': dall_e.state_dict(),

              'pt_opt': opt.state_dict(),

              'pt_sch': sch.state_dict() }
    
    torch.save(save_obj, path)

In [ ]:
#dall_e = DALLE(**parameters,vae = VQGanVAE()).to(device) ## use it only for first epoch 

pathpt="/kaggle/input/newwpoch3/dalle_ep3.pt"   ##### dont use for first epoch 

ptload=torch.load(pathpt)                              ####  dont use for first epoch 

pt_params=ptload["hparams"]                              ####  dont use for first epoch 
pt_weights=ptload["weights"]                              ####  dont use for first epoch 
pt_opt=ptload.get("pt_opt")                               ####  dont use for first epoch 
pt_sch=ptload.get("pt_sch")                               ####  dont use for first epoch 

dall_e=DALLE(**pt_params,vae = VQGanVAE()).to(device)  #### dont use for first epoch 
dall_e.load_state_dict(pt_weights)                    #### dont use for first epoch 

opt = torch.optim.Adam(dall_e.parameters(), lr=0.001)
opt.load_state_dict(pt_opt)                              ####  dont use for first epoch 


sch = torch.optim.lr_scheduler.ExponentialLR(opt, 0.98)
sch.load_state_dict(pt_sch)                              ####  dont use for first epoch 


In [ ]:
words = pd.DataFrame(word_code)
words = words.sort_values(0, axis=1)
array_of_words = np.asarray(words.columns)
word_dict = {}
index_dict = {}

for i in range(len(array_of_words)):
 w = array_of_words[i]
 word_dict[w] = i
 index_dict[i] = w



In [ ]:
index=443 ##  change it for different captions

capt = torch.tensor(test_cap2[index]).unsqueeze(dim=0).to(device)

images = dall_e.generate_images(capt)

plt.axis("off")
plt.imshow( images.cpu().detach().squeeze().permute(1, 2, 0) )


print(test_url[test_imid2[index]])
sentence=[]

 for i in test_cap2[index]:

 sentence.append(index_dict[i])

print(sentence)